# Dual LP

## 1. Construction of the Dual to an LP

Your first task will be to implement a function that constructs the dual to an LP in canonical form.

### Example of Dual

As seen in the script the *primal* LP:
$$\begin{equation}
\begin{array}
\\max \ z = & 4x_1 & + & x_2  & + & 5x_3 & + & 3x_4, &      &   \\
s.t:     & x_1  & - & x_2  & - & x_3  & + & 3x_4  & \leq & 1 \\
         & 5x_1 & + & x_2  & + & 3x_3 & + & 8x_4  & \leq & 55\\
         & -x_1 & + & 2x_2 & + & 3x_3 & - & 5x_4  & \leq & 3 \\
\end{array}\\
        x_1\geq 0,x_2\geq 0,x_3\geq 0,x_4\geq 0.
\end{equation}$$

Will become the following *dual problem*:
$$\begin{equation}
\begin{array}
\\min \ w = & y_1  & + & 55y_2 & + & 3y_3 &      &   \\
s.t:        & y_1  & + & 5y_2  & - & y_3  & \geq & 4 \\
            & -y_1 & + & y_2   & + & 2y_3 & \geq & 1 \\
            & -y_1 & + & 3y_2  & + & 3y_3 & \geq & 5 \\
            & 3y_1 & + & 8y_2  & - & 5y_3 & \geq & 3 \\
\end{array}\\
        y_1\geq 0,y_2\geq 0,y_3\geq 0.
\end{equation}$$

<br><br>
To solve the Dual with the simplex method we implemented, we still have to convert the Dual into canonical form. This is easily achieved. The minimization of the objective function can be seen as the maximization of the negative objective function (min$(w)=$max$(-w)$). Additional we need to change the constraints to have 'lesser-than or equal' signs. <br>So in summary we get to the canonical form by multiplying each equation with $-1$. <br>
If you look at the resulting tableaus you see, that we achieve the Dual by rotating rows and moving them to the right place.

`Task:` Implement a function that converts a primal tableau `P` to the tableau `D` of the corresponding Dual in canonical form.

In [1]:
import numpy as np
P = np.matrix([
    [-4,-1,-5,-3, 0],
    [1,-1,-1, 3, 1],
    [5, 1, 3, 8, 55],
    [-1,2, 3,-5, 3]
])

def toDual(P):
    n=P.shape[0]
    m=P.shape[1]
    D = np.zeros(shape=(m,n))
    for i in range(0,n):
        D[1:m,i-1] = P[i,:n]
    for i in range(0,n-1):
        D[0,i]=P[i+1,m-1]
    D = D * -1
    return D

print(toDual(P))

[[ -1. -55.  -3.  -0.]
 [ -1.  -5.   1.   4.]
 [  1.  -1.  -2.   1.]
 [  1.  -3.  -3.   5.]
 [ -3.  -8.   5.   3.]]


## 2. Computation of optimal solutions for primal and dual LPs



### 2.1 Example 1
Given the following LP compute the primal and dual solution.

$$\begin{equation}
\begin{array}
\\max \ z = & 4x_1 & + & 9x_2 & + & 2x_3 & + &  x_4, &      &   \\
s.t:        & x_1  & + & 5x_2 & + & 7x_3 & - & 2x_4  & \leq & -4\\
            & 3x_1 & + & 7x_2 & + & 5x_3 & + & 4x_4  & \leq &  9\\
            & 3x_1 & + & 3x_2 & + & 12x_3& - & 5x_4  & \leq & 20\\
\end{array}\\
        x_1\geq 0,x_2\geq 0,x_3\geq 0,x_4\geq 0.
\end{equation}$$

To solve the problems, use either your own Simplex-Method from last week or compute the optimal solutions using pulp.

The corresponding dual is:
$$\begin{equation}
\begin{array}
\\min \ w = & -4y_1 & + & 9y_2  & + & 20y_3 &      &   \\
s.t:        & y_1   & + & 3y_2  & + & 3y_3  & \geq & 4 \\
            & 5y_1  & + & 7y_2  & + & 3y_3  & \geq & 9 \\
            & 7y_1  & + & 5y_2  & + & 12y_3 & \geq & 2 \\
            & -2y_1 & + & 4y_2  & - & 5y_3  & \geq & 1 \\
\end{array}\\
        y_1\geq 0,y_2\geq 0,y_3\geq0.
\end{equation}$$

In [2]:
import pulp

#Create the LP
primalLP1 = pulp.LpProblem("Primal1", pulp.LpMaximize)

#Create the variables
x1 = pulp.LpVariable('x1', lowBound=0, cat=pulp.LpContinuous)
x2 = pulp.LpVariable('x2', lowBound=0, cat=pulp.LpContinuous)
x3 = pulp.LpVariable('x3', lowBound=0, cat=pulp.LpContinuous)
x4 = pulp.LpVariable('x4', lowBound=0, cat=pulp.LpContinuous)

#Add the objective function
primalLP1 += 4*x1 + 9*x2 + 2*x3 + x4

#Add the constraints
primalLP1 += x1 + 5*x2 + 7*x3 - 2*x4 <= -4
primalLP1 += 3*x1 + 7*x2 + 5*x3 + 4*x4  <= 9
primalLP1 += 3*x1 + 3*x2 + 12*x3 - 5*x4 <= 20

primalLP1.solve()
print("Solution of primal:")
#Print out the optimal objective value of primalLP
print(primalLP1.objective.value())
#Print out optimal vertex.
for v in primalLP1.variables():
    print(str(v) + " = " + str(v.value()))
print("Status: " + str( pulp.LpStatus[primalLP1.status]) )

#Create the LP
dualLP1 = pulp.LpProblem("Dual1", pulp.LpMinimize)

#Create the variables
y1 = pulp.LpVariable('y1', lowBound=0, cat=pulp.LpContinuous)
y2 = pulp.LpVariable('y2', lowBound=0, cat=pulp.LpContinuous)
y3 = pulp.LpVariable('y3', lowBound=0, cat=pulp.LpContinuous)


#Add the objective function
dualLP1 += -4*y1 + 9*y2 + 20*y3

#Add the constraints
dualLP1 += y1 + 3*y2 + 3*y3 >=4
dualLP1 += 5*y1 + 7*y2 + 3*y3 >=9
dualLP1 += 7*y1 + 5*y2 + 12*y3 >=2
dualLP1 += -2*y1 + 4*y2 - 5*y3 >=1

dualLP1.solve()
print("Solution of dual:")
#Print out the optimal objective value of primalLP
print(dualLP1.objective.value())
#Print out optimal vertex.
for v in dualLP1.variables():
    print(str(v) + " = " + str(v.value()))
print("Status: " + str( pulp.LpStatus[dualLP1.status]) )

Solution of primal:
2.9000000000000004
x1 = 0.2
x2 = 0.0
x3 = 0.0
x4 = 2.1
Status: Optimal
Solution of dual:
2.8999999999999995
y1 = 1.3
y2 = 0.9
y3 = 0.0
Status: Optimal


Note: The small difference in the optimal value was caused by bad rounding

In [3]:
#Optional Task:

#Create the LP
primalLP1 = pulp.LpProblem("Primal1", pulp.LpMaximize)
dualLP1 = pulp.LpProblem("Dual1", pulp.LpMaximize)

#Create the variables
#If you want to automate this you can use dictionaries, I will just define it
x1 = pulp.LpVariable('x1', lowBound=0, cat=pulp.LpContinuous)
x2 = pulp.LpVariable('x2', lowBound=0, cat=pulp.LpContinuous)
x3 = pulp.LpVariable('x3', lowBound=0, cat=pulp.LpContinuous)
x4 = pulp.LpVariable('x4', lowBound=0, cat=pulp.LpContinuous)
y1 = pulp.LpVariable('y1', lowBound=0, cat=pulp.LpContinuous)
y2 = pulp.LpVariable('y2', lowBound=0, cat=pulp.LpContinuous)
y3 = pulp.LpVariable('y3', lowBound=0, cat=pulp.LpContinuous)

#Tableau
P = np.matrix([
    [-4,-9,-2,-1, 0],
    [1 , 5, 7,-2,-4],
    [3 , 7, 5, 4, 9],
    [3 , 3,12,-5,20]
])
D = toDual(P)

obj = True
for row in P:
    if (obj==True):
        primalLP1 += -row.item(0)*x1-row.item(1)*x2-row.item(2)*x3-row.item(3)*x4
        obj=False
    else:
        primalLP1 += row.item(0)*x1+row.item(1)*x2+row.item(2)*x3+row.item(3)*x4 <= row.item(4)


obj = True
for row in D:
    if (obj==True):
        dualLP1 += -row.item(0)*y1-row.item(1)*y2-row.item(2)*y3
        obj=False
    else:
        dualLP1 += row.item(0)*y1+row.item(1)*y2+row.item(2)*y3 <= row.item(3)



primalLP1.solve()
print("Solution of primal:")
#Print out the optimal objective value of primalLP
print(primalLP1.objective.value())
#Print out optimal vertex.
for v in primalLP1.variables():
    print(str(v) + " = " + str(v.value()))
print("Status: " + str( pulp.LpStatus[primalLP1.status]) )
dualLP1.solve()
print("Solution of dual:")
#Print out the optimal objective value of primalLP
print(dualLP1.objective.value())
#Print out optimal vertex.
for v in dualLP1.variables():
    print(str(v) + " = " + str(v.value()))
print("Status: " + str( pulp.LpStatus[dualLP1.status]) )

Solution of primal:
2.9000000000000004
x1 = 0.2
x2 = 0.0
x3 = 0.0
x4 = 2.1
Status: Optimal
Solution of dual:
4.0
y1 = 0.0
y2 = 0.0
y3 = 0.2
Status: Unbounded


In the following examples we want to check if our results are in accordance with what we have seen in the script, focussing primarily on `Figure 3.1`.

### 2.2 Example 2
Given the following LP compute the primal and dual solution.

$$\begin{equation}
\begin{array}
\\max \ z = & 4x_1 & + & 9x_2 & + & 2x_3 & + &  x_4, &      &   \\
s.t:        & x_1  & + & 5x_2 & + & 7x_3 & + & 2x_4  & \leq & -4\\
            & 3x_1 & + & 7x_2 & + & 5x_3 &   &       & \leq &  9\\
            & 3x_1 & + & 3x_2 & + & 12x_3& + & 5x_4  & \leq & 20\\
\end{array}\\
        x_1\geq 0,x_2\geq 0,x_3\geq 0,x_4\geq 0.
\end{equation}$$

To solve the problems, use either your own Simplex-Method from last week or compute the optimal solutions using pulp.

The corresponding dual is:
$$\begin{equation}
\begin{array}
\\min \ w = & -4y_1 & + & 9y_2  & + & 20y_3 &      &   \\
s.t:        & y_1   & + & 3y_2  & + & 3y_3  & \leq & 4 \\
            & 5y_1  & + & 7y_2  & + & 3y_3  & \leq & 9 \\
            & 7y_1  & + & 5y_2  & + & 12y_3 & \leq & 2 \\
            & 2y_1  &   &       & + & 5y_3  & \leq & 1 \\
\end{array}\\
        y_1\geq 0,y_2\geq 0,y_3\geq0.
\end{equation}$$

In [4]:
import pulp

#Create the LP
primalLP2 = pulp.LpProblem("Primal2", pulp.LpMaximize)

#Create the variables
x1 = pulp.LpVariable('x1', lowBound=0, cat=pulp.LpContinuous)
x2 = pulp.LpVariable('x2', lowBound=0, cat=pulp.LpContinuous)
x3 = pulp.LpVariable('x3', lowBound=0, cat=pulp.LpContinuous)
x4 = pulp.LpVariable('x4', lowBound=0, cat=pulp.LpContinuous)

#Add the objective function
primalLP2 += 4*x1 + 9*x2 + 2*x3 + x4

#Add the constraints
primalLP2 += x1 + 5*x2 + 7*x3 + 2*x4 <= -4
primalLP2 += 3*x1 + 7*x2 + 5*x3  <= 9
primalLP2 += 3*x1 + 3*x2 + 12*x3 + 5*x4 <= 20

primalLP2.solve()
print("Solution of primal:")
print("Status: " + str( pulp.LpStatus[primalLP2.status]) )

#Create the LP
dualLP2 = pulp.LpProblem("Dual2", pulp.LpMinimize)

#Create the variables
y1 = pulp.LpVariable('y1', lowBound=0, cat=pulp.LpContinuous)
y2 = pulp.LpVariable('y2', lowBound=0, cat=pulp.LpContinuous)
y3 = pulp.LpVariable('y3', lowBound=0, cat=pulp.LpContinuous)


#Add the objective function
dualLP2 += -4*y1 + 9*y2 + 20*y3

#Add the constraints
dualLP2 += y1 + 3*y2 + 3*y3 >=4
dualLP2 += 5*y1 + 7*y2 + 3*y3 >=9
dualLP2 += 7*y1 + 5*y2 + 12*y3 >=2
dualLP2 += 2*y1          + 5*y3 >=1

dualLP2.solve()
print("Solution of dual:")
print("Status: " + str( pulp.LpStatus[dualLP2.status]) )

Solution of primal:
Status: Infeasible
Solution of dual:
Status: Unbounded


### 2.3 Example 3
Given the following LP compute the primal and dual solution.

$$\begin{equation}
\begin{array}
\\max \ z = & 2x_1 & - &  x_2 & - &  x_3 & - & 8x_4, &      &   \\
s.t:        & x_1  & - &  x_2 & - &  x_3 & - &  x_4  & \leq &  1\\
            & -1x_1& + &  x_2 & + & 3x_3 & + & 4x_4  & \leq & -9\\
            & 15x_1& + & 2x_2 & - &100x_3& + & 3x_4  & \leq &-20\\
\end{array}\\
        x_1\geq 0,x_2\geq 0,x_3\geq 0,x_4\geq 0.
\end{equation}$$

To solve the problems, use either your own Simplex-Method from last week or compute the optimal solutions using pulp.

The corresponding dual is:
$$\begin{equation}
\begin{array}
\\min \ w = &   y_1  & - & 9y_2  & - & 20y_3 &      &   \\
s.t:        & y_1    & - &  y_2  & + & 15y_3 & \geq & 2 \\
            & -1y_1  & + &  y_2  & + & 2y_3  & \geq &-1 \\
            & -1y_1  & + & 3y_2  & - &100y_3 & \geq &-1 \\
            & -1y_1  & + & 4y_2  & + & 3y_3  & \geq &-8 \\
\end{array}\\
        y_1\geq 0,y_2\geq 0,y_3\geq0.
\end{equation}$$

In [5]:
import pulp

#Create the LP
primalLP3 = pulp.LpProblem("Primal3", pulp.LpMaximize)

#Create the variables
x1 = pulp.LpVariable('x1', lowBound=0, cat=pulp.LpContinuous)
x2 = pulp.LpVariable('x2', lowBound=0, cat=pulp.LpContinuous)
x3 = pulp.LpVariable('x3', lowBound=0, cat=pulp.LpContinuous)
x4 = pulp.LpVariable('x4', lowBound=0, cat=pulp.LpContinuous)

#Add the objective function
primalLP3 += 2*x1 - x2 - x3 - 8*x4

#Add the constraints
primalLP3 +=  x1   - x2 -  x3 -  x4  <= 1
primalLP3 += -1*x1 + x2 + 3*x3 + 4*x4 <= -9
primalLP3 +=  15*x1 + 2*x2 -100*x3 + 3*x4 <= -20

primalLP3.solve()
print("Solution of primal:")
print("Status: " + str( pulp.LpStatus[primalLP3.status]) )

#Create the LP
dualLP3 = pulp.LpProblem("Dual3", pulp.LpMinimize)

#Create the variables
y1 = pulp.LpVariable('y1', lowBound=0, cat=pulp.LpContinuous)
y2 = pulp.LpVariable('y2', lowBound=0, cat=pulp.LpContinuous)
y3 = pulp.LpVariable('y3', lowBound=0, cat=pulp.LpContinuous)


#Add the objective function
dualLP3 += -4*y1 - 9*y2 - 20*y3

#Add the constraints
dualLP3 += y1    -   y2 + 15*y3  >=2
dualLP3 += -1*y1 +   y2 + 2*y3   >=-1
dualLP3 += -1*y1 + 3*y3 - 100*y3 >=-1
dualLP3 += -1*y1 + 4*y2 + 3*y3   >=-8

dualLP3.solve()
print("Solution of dual:")
print("Status: " + str( pulp.LpStatus[dualLP3.status]) )

Solution of primal:
Status: Infeasible
Solution of dual:
Status: Infeasible


### 2.4 Example 4
Given the following LP compute the primal and dual solution.

$$\begin{equation}
\begin{array}
\\max \ z = & 4x_1 & + & 9x_2 & + & 2x_3 & + &  x_4, &      &   \\
s.t:        & x_1  & + & 5x_2 & - & 7x_3 & + & 2x_4  & \leq & 4 \\
            & 3x_1 & + & 7x_2 & - & 5x_3 &   &       & \leq & 20\\
            & -3x_1& + & 3x_2 & + & 12x_3& - & 5x_4  & \leq & 8 \\
\end{array}\\
        x_1\geq 0,x_2\geq 0,x_3\geq 0,x_4\geq 0.
\end{equation}$$

To solve the problems, use either your own Simplex-Method from last week or compute the optimal solutions using pulp.

The corresponding dual is:
$$\begin{equation}
\begin{array}
\\min \ w = & 4y_1  & + & 20y_2 & + & 8y_3  &      &   \\
s.t:        & y_1   & + & 3y_2  & - & 3y_3  & \geq & 4 \\
            & 5y_1  & + & 7y_2  & + & 3y_3  & \geq & 9 \\
            & -7y_1 & - & 5y_2  & + & 12y_3 & \geq & 2 \\
            & 2y_1  &   &       & - & 5y_3 &  \geq & 1 \\
\end{array}\\
        y_1\geq 0,y_2\geq 0,y_3\geq0.
\end{equation}$$

In [6]:
import pulp

#Create the LP
primalLP4 = pulp.LpProblem("Primal4", pulp.LpMaximize)

#Create the variables
x1 = pulp.LpVariable('x1', lowBound=0, cat=pulp.LpContinuous)
x2 = pulp.LpVariable('x2', lowBound=0, cat=pulp.LpContinuous)
x3 = pulp.LpVariable('x3', lowBound=0, cat=pulp.LpContinuous)
x4 = pulp.LpVariable('x4', lowBound=0, cat=pulp.LpContinuous)

#Add the objective function
primalLP4 += 4*x1 + 9*x2 + 2*x3 + x4

#Add the constraints
primalLP4 += x1 + 5*x2 - 7*x3 + 2*x4 <= 4
primalLP4 += 3*x1 + 7*x2 - 5*x3  <= 20
primalLP4 += -3*x1 + 3*x2 + 12*x3 - 5*x4 <= 8

primalLP4.solve()
print("Solution of primal:")
print("Status: " + str( pulp.LpStatus[primalLP4.status]) )

#Create the LP
dualLP4 = pulp.LpProblem("Dual4", pulp.LpMinimize)

#Create the variables
y1 = pulp.LpVariable('y1', lowBound=0, cat=pulp.LpContinuous)
y2 = pulp.LpVariable('y2', lowBound=0, cat=pulp.LpContinuous)
y3 = pulp.LpVariable('y3', lowBound=0, cat=pulp.LpContinuous)


#Add the objective function
dualLP4 += 4*y1 + 20*y2 + 8*y3

#Add the constraints
dualLP4 += y1 + 3*y2 - 3*y3 >=4
dualLP4 += 5*y1 + 7*y2 + 3*y3 >=9
dualLP4 += -7*y1 - 5*y2 + 12*y3 >=2
dualLP4 += 2*y1          - 5*y3 >=1

dualLP4.solve()
print("Solution of dual:")
print("Status: " + str( pulp.LpStatus[dualLP4.status]) )

Solution of primal:
Status: Unbounded
Solution of dual:
Status: Infeasible


### 2.5
Compare your results with `Figure 3.1` from the script.

The results of our 4 examples are in accordance with the table in the script.